In [1]:
import os
import json
from dotenv import load_dotenv
from pathlib import Path

dotenv_path = Path("../.env")
if dotenv_path.exists():
    print("Loaded .env file!")
    load_dotenv(str(dotenv_path))

Loaded .env file!


In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct", device_map=0, torch_dtype=torch.float16)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
messages = [
    {"role": "system", "content": "You are an expert at finding identification numbers."},
    {"role": "user", "content": """Text:\nPre- Induction Assignment 3 MediPath  #ItGetsBetter

Anil Dubey

276795361801

Customer Segments Unfair advantage

• Search Engine  Optimization

• Social Media Marketing

• Influencer marketing

• Mobile booths

• Billboards

#itgetsbetter

Channels

Unique Value Proposition

Create a brand uniquely

associated with mental

health

Showcase sample audio

therapy sessions on
     

In the above text, is 276795361801 more likely to be an identification number or a phone number?
     
A. Identification number
B. Phone number
"""},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [4]:
outputs = model.generate(
    input_ids,
    max_new_tokens=1,
    eos_token_id=terminators,
    do_sample=False,
    output_scores=True,
    return_dict_in_generate =True
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [9]:
tokenizer.batch_decode(outputs, skip_special_tokens=True)

['system\n\nYou are an expert at finding identification numbers.user\n\nText:\nPre- Induction Assignment 3 MediPath  #ItGetsBetter\n\nAnil Dubey\n\n276795361801\n\nCustomer Segments Unfair advantage\n\n• Search Engine  Optimization\n\n• Social Media Marketing\n\n• Influencer marketing\n\n• Mobile booths\n\n• Billboards\n\n#itgetsbetter\n\nChannels\n\nUnique Value Proposition\n\nCreate a brand uniquely\n\nassociated with mental\n\nhealth\n\nShowcase sample audio\n\ntherapy sessions on\n     \n\nIn the above text, is 276795361801 more likely to be an identification number or a phone number?\n     \nA. Identification number\nB. Phone numberassistant\n\nBased']

In [15]:
outputs["scores"][0].argsort()

tensor([[  779,   393,   328,  ...,    32,    40, 29815]], device='cuda:0')

In [18]:
tokenizer("\nB")

{'input_ids': [198, 33], 'attention_mask': [1, 1]}

In [19]:
tokenizer("\nA")

{'input_ids': [198, 32], 'attention_mask': [1, 1]}

In [6]:
outputs["scores"][0][0, [32, 33]]

tensor([21.1562, 15.7109], device='cuda:0')

In [9]:
messages = [
    {"role": "system", "content": "You are an expert at finding identification numbers."},
    {"role": "user", "content": """Text:\nTechnical and Artistic Theatre and Performing Art division

Z.S. 30407059

Challenge

Working in the Technical and Artistic 
     
     
In the above text, list all identification numbers and phone numbers.
"""}
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

outputs = model.generate(
    input_ids,
    max_new_tokens=20,
    eos_token_id=terminators,
    do_sample=False,
)

tokenizer.batch_decode(outputs, skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['system\n\nYou are an expert at finding identification numbers.user\n\nText:\nTechnical and Artistic Theatre and Performing Art division\n\nZ.S. 30407059\n\nChallenge\n\nWorking in the Technical and Artistic \n     \n     \nIn the above text, list all identification numbers and phone numbers.assistant\n\nBased on the provided text, I found one identification number:\n\n* Z.S. 30407059']

In [6]:
text = """Reflection – Visualization   Deiby\n\nChallenge    For some time I frequented a public gym and showered there before my work day.  Although the  showers may have been cleaned daily, they were not cleaned between each use - sometimes  the floor was, well “icky” and traditional flip-flops just didn’t cut it.  I searched and searched, for  an existing product that would provide the protection I needed, without any success.  I then  realized a need for a solution for increased sanitation when using any facility that has communal  showers - gyms, dorms, camps, athletic training facilities, etc."""

messages = [
    {"role": "system", "content": "You are an expert at finding human person names."},
    {"role": "user", "content": text + "\n\nIn the above text, is `Deiby` a human person name? Yes or No?"}
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

outputs = model.generate(
    input_ids,
    max_new_tokens=20,
    eos_token_id=terminators,
    do_sample=False,
)

tokenizer.batch_decode(outputs, skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['system\n\nYou are an expert at finding human person names.user\n\nReflection – Visualization   Deiby\n\nChallenge    For some time I frequented a public gym and showered there before my work day.  Although the  showers may have been cleaned daily, they were not cleaned between each use - sometimes  the floor was, well “icky” and traditional flip-flops just didn’t cut it.  I searched and searched, for  an existing product that would provide the protection I needed, without any success.  I then  realized a need for a solution for increased sanitation when using any facility that has communal  showers - gyms, dorms, camps, athletic training facilities, etc.\n\nIn the above text, is `Deiby` a human person name? Yes or No?assistant\n\nYes.']

In [8]:
text = """
Name: Bk Bk  Roll No: 172801513686

1 | P a g e

STORY TELLING

Now-a-days the usage of technology is at the next level. All of  us are very much interested in using the present technology  and making their work easy and perfectly. The usage of  technology has many advantages and disadvantages based on  our usage it is helpful.

In present days the people in our society are very busy through  their daily works and forget their things and security of others,  even I forget many things which are not visible at the right time.  Now coming to the security now-a-days there is more of  increasing of women trafficking and rapes and kidnapping and  hence blackmailing their parents for money and other issues.

So to get a solution for this, I had introduced a GPS TRAKIN  app which is connected to different gadgets that work for all  users, like the things which are stolen and also forgotten; it is  used to locate your pets nearby, and in farming, security, track  you family and friends location.

Name: Bk Bk  Roll No: 172801513686

2 | P a g e

User

Queries

Meaning

Conclusion

Sama    208798413907

Why did you  come up with  this idea?

To introduce  the type of  technology  which can be  used in our  daily-to-daily  life.

To make the  awareness and  maintain their  personal security  and to locate their  forgotten things  and pets.

Md    347376430553

How this is  used in  farming?

Farmers can  put the GPS  receivers on  their tractors  and other  framing  equipment.  This allows  them to map  their  plantations  and return to  same time.

Hence it is  suitable for use in  mapping soil  sample locations,  and the farmer  can locate the  areas that have  soil suitable for  framing.

Giuseppe    943995368223

Are these  gadgets are  affordable?

Yes, it is  affordable for  all people and  depends upon  their usage.

Hence the app  interface is also  so simple and  can  understandable  for uneducated  people also."""

messages = [
    {"role": "system", "content": "You are an expert at finding human person names."},
    {"role": "user", "content": text + "\n\nIn the above text, list all human person names."}
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

outputs = model.generate(
    input_ids,
    max_new_tokens=100,
    eos_token_id=terminators,
    do_sample=False,
)

tokenizer.batch_decode(outputs, skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['system\n\nYou are an expert at finding human person names.user\n\nName: Bk Bk  Roll No: 172801513686\n\n1 | P a g e\n\nSTORY TELLING\n\nNow-a-days the usage of technology is at the next level. All of  us are very much interested in using the present technology  and making their work easy and perfectly. The usage of  technology has many advantages and disadvantages based on  our usage it is helpful.\n\nIn present days the people in our society are very busy through  their daily works and forget their things and security of others,  even I forget many things which are not visible at the right time.  Now coming to the security now-a-days there is more of  increasing of women trafficking and rapes and kidnapping and  hence blackmailing their parents for money and other issues.\n\nSo to get a solution for this, I had introduced a GPS TRAKIN  app which is connected to different gadgets that work for all  users, like the things which are stolen and also forgotten; it is  used to locate your

In [9]:
messages = [
    # {"role": "system", "content": "You are an expert at finding human person names."},
    {"role": "user", "content": text + "\n\nIn the above text, list all identification numbers."}
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

outputs = model.generate(
    input_ids,
    max_new_tokens=100,
    eos_token_id=terminators,
    do_sample=False,
)

tokenizer.batch_decode(outputs, skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['user\n\nName: Bk Bk  Roll No: 172801513686\n\n1 | P a g e\n\nSTORY TELLING\n\nNow-a-days the usage of technology is at the next level. All of  us are very much interested in using the present technology  and making their work easy and perfectly. The usage of  technology has many advantages and disadvantages based on  our usage it is helpful.\n\nIn present days the people in our society are very busy through  their daily works and forget their things and security of others,  even I forget many things which are not visible at the right time.  Now coming to the security now-a-days there is more of  increasing of women trafficking and rapes and kidnapping and  hence blackmailing their parents for money and other issues.\n\nSo to get a solution for this, I had introduced a GPS TRAKIN  app which is connected to different gadgets that work for all  users, like the things which are stolen and also forgotten; it is  used to locate your pets nearby, and in farming, security, track  you family 

In [10]:
text = """Reflection paper of Art Street – Mind Mapping

Challenge    I’m a 5th year Electronics Engineering student in school, a youth leader to our home parish which is

St. Rose of Lima Parish at Teresa, Rizal, and I can say that I’m a friendly person, because I have lots of  friends. In studying my lectures, doing my projects, guiding youth, and becoming a friend and big brother to  my siblings which requires so many attention, and decisions to make. In line with this, I encounter many  problems and challenges regarding decision making and thinking about the best idea that I could think of. In  my journey to become an Engineer, I find it hard to think of the best idea. I have lots of group work, laboratory  activities, and projects that need to be done by team, and most of the time, because we have different minds  and different thinking, we can’t agree on the best idea that we can have. In my experience as a youth leader,  as an individual, I must touch every life of each youth that I’m handling and inspire them to do more, and to  become better in life. As I coach them or give them advice, my always question in myself is that “How I can  do it in different but effective way?”, and “Does my advice help them to be motivated?” And the hardest role  that I can be is to be a friend to others, and to be the big brother to my siblings. It is because every detail that  I will tell to them, every story that I will share, and every idea that I will feed to them, is must be accurate,  real, and which can boost their courage to step forward in life. Because every word I say, it can affect the  path of life that will be taking.    Selection    As what I’ve learn, mind mapping is a tool that you will think of a subject and its idea and sub-idea. I
"""

messages = [
    {"role": "system", "content": "You are an expert at finding human person names."},
    {"role": "user", "content": text + "\n\nIn the above text, list all person names."}
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

outputs = model.generate(
    input_ids,
    max_new_tokens=100,
    eos_token_id=terminators,
    do_sample=False,
)

tokenizer.batch_decode(outputs, skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['system\n\nYou are an expert at finding human person names.user\n\nReflection paper of Art Street – Mind Mapping\n\nChallenge    I’m a 5th year Electronics Engineering student in school, a youth leader to our home parish which is\n\nSt. Rose of Lima Parish at Teresa, Rizal, and I can say that I’m a friendly person, because I have lots of  friends. In studying my lectures, doing my projects, guiding youth, and becoming a friend and big brother to  my siblings which requires so many attention, and decisions to make. In line with this, I encounter many  problems and challenges regarding decision making and thinking about the best idea that I could think of. In  my journey to become an Engineer, I find it hard to think of the best idea. I have lots of group work, laboratory  activities, and projects that need to be done by team, and most of the time, because we have different minds  and different thinking, we can’t agree on the best idea that we can have. In my experience as a youth leade

In [11]:
text = """Reflection paper of Art Street – Mind Mapping

Challenge    I’m a 5th year Electronics Engineering student in school, a youth leader to our home parish which is

St. Rose of Lima Parish at Teresa, Rizal, and I can say that I’m a friendly person, because I have lots of  friends. In studying my lectures, doing my projects, guiding youth, and becoming a friend and big brother to  my siblings which requires so many attention, and decisions to make. In line with this, I encounter many  problems and challenges regarding decision making and thinking about the best idea that I could think of. In  my journey to become an Engineer, I find it hard to think of the best idea. I have lots of group work, laboratory  activities, and projects that need to be done by team, and most of the time, because we have different minds  and different thinking, we can’t agree on the best idea that we can have. In my experience as a youth leader,  as an individual, I must touch every life of each youth that I’m handling and inspire them to do more, and to  become better in life. As I coach them or give them advice, my always question in myself is that “How I can  do it in different but effective way?”, and “Does my advice help them to be motivated?” And the hardest role  that I can be is to be a friend to others, and to be the big brother to my siblings. It is because every detail that  I will tell to them, every story that I will share, and every idea that I will feed to them, is must be accurate,  real, and which can boost their courage to step forward in life. Because every word I say, it can affect the  path of life that will be taking.    Selection    As what I’ve learn, mind mapping is a tool that you will think of a subject and its idea and sub-idea. I
"""

messages = [
    {"role": "system", "content": "You are an expert at finding human person names."},
    {"role": "user", "content": text + "\n\nIn the above text is `Art Street` a person name?"}
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

outputs = model.generate(
    input_ids,
    max_new_tokens=100,
    eos_token_id=terminators,
    do_sample=False,
)

tokenizer.batch_decode(outputs, skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['system\n\nYou are an expert at finding human person names.user\n\nReflection paper of Art Street – Mind Mapping\n\nChallenge    I’m a 5th year Electronics Engineering student in school, a youth leader to our home parish which is\n\nSt. Rose of Lima Parish at Teresa, Rizal, and I can say that I’m a friendly person, because I have lots of  friends. In studying my lectures, doing my projects, guiding youth, and becoming a friend and big brother to  my siblings which requires so many attention, and decisions to make. In line with this, I encounter many  problems and challenges regarding decision making and thinking about the best idea that I could think of. In  my journey to become an Engineer, I find it hard to think of the best idea. I have lots of group work, laboratory  activities, and projects that need to be done by team, and most of the time, because we have different minds  and different thinking, we can’t agree on the best idea that we can have. In my experience as a youth leade